In [1]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import json
import git


In [2]:
# get commit number for github repository
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
print(sha)

d353e5f3c747a9d747afb3934f418586cd78de17


In [3]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',
    'full_diagonalization' : True,
    'number_of_photons' : 10, 
}




In [6]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : "https://github.com/mapol-chem/qed-ci",
      "branch" : "photon_number_basis",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "LiH",
    "geometry": [],
    "symbols": [
      "Li",
      "H"
    ],
    "z-matrix" : mol_str,
  },
    "driver": "energy",
    "model" : {
        "method" : "qed-fci",
        "orbital_basis"  : "sto-3g",
        "photon_basis" : "number_basis",
        "number_of_photon_states" : cavity_dict["number_of_photons"],
        "lambda" : [
            cavity_dict["lambda_vector"][0],
            cavity_dict["lambda_vector"][1],
            cavity_dict["lambda_vector"][2]
        ],
        "omega" : cavity_dict["omega_value"]  
    },
    
    "return_result" : [
        
    ],
    
    "cavity_free_rhf_energy" : 0.,
    "qed_rhf_energy" : 0.,
    "cavity_free_fci_ground_state_energy" : 0.,
}

# function to generate file names based on system details
def generate_file_name(dic, r):
    string_r = "{:.3f}".format(r)
    file_name = dic["molecule"]["molecule_name"] + "_"
    file_name += "r_" + string_r + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += str(dictionary["model"]["photon_basis"]) + "_"
    file_name += str(dictionary["model"]["number_of_photon_states"]) + "_"
    file_name += "lambda_" + str(dictionary["model"]["lambda"][0]) 
    file_name += "_" + str(dictionary["model"]["lambda"][1]) 
    file_name += "_" + str(dictionary["model"]["lambda"][2]) + "_"
    file_name += "omega_" + str(dictionary["model"]["omega"]) + ".json"
    return file_name

In [7]:
# print a test dictionary and filename
print(dictionary)
print(generate_file_name(dictionary, 1.833443))

{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.6775510204081634\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result': [], 'cavity_free_rhf_energy': 0.0, 'qed_rhf_energy': 0.0, 'cavity_free_fci_ground_state_energy': 0.0}
LiH_r_1.833_qed-fci_sto-3g_number_basis_10_lambda_0.0_0.0_0.05_omega_0.12086.json


In [8]:
# scan through bond-stretch and write .json file for each geometry
mol_tmpl = """
Li
H 1 **R**
symmetry c1
"""

mol_str = """
Li
H 1 1.5
symmetry c1
"""

N_R = 50
N_el = 50
r_array = np.linspace(1.4, 2.2, N_R)

E_array = np.zeros((N_R, N_el))

r_idx = 0
for r in r_array:
    file_name = generate_file_name(dictionary, r) 
    mol_str = mol_tmpl.replace("**R**", str(r))
    mol = psi4.geometry(mol_str)
    #psi4.set_options({'restricted_docc': [3],'active': [4],'num_roots':2})
    scf_e, wfn = psi4.energy('SCF/sto-3g', return_wfn=True)
    fci_energy = psi4.energy('fci',ref_wfn=wfn)    
    LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_dict)
    
    dictionary["return_result"] = [LiH.CIeigs[0], LiH.CIeigs[1], LiH.CIeigs[2], LiH.CIeigs[3], LiH.CIeigs[4], LiH.CIeigs[5], LiH.CIeigs[6]]
    dictionary["cavity_free_rhf_energy"] = scf_e
    dictionary["cavity_free_fci_ground_state_energy"] = fci_energy
    geometry = np.asarray(mol.geometry())
    geometry_list = geometry.tolist()
    dictionary["molecule"]["geometry"] = geometry_list
    dictionary["molecule"]["z-matrix"] = mol_str
    E_array[r_idx, :] = LiH.CIeigs[:50]
    r_idx += 1
    print(dictionary)
    json_object = json.dumps(dictionary, indent=4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)



 Completed QED-RHF in 0.08571028709411621 seconds
 Completed 1HSO Build in 2.6941299438476562e-05 seconds
 Completed ERI Build in 0.0012340545654296875 seconds 
 Completed 2D build in 0.0001049041748046875 seconds
 Completed 1G build in 1.4066696166992188e-05 seconds
 Completed determinant list in 0.00043773651123046875 seconds 
 Completed constant offset matrix in 0.00015401840209960938 seconds
 Completed Hamiltonian build in 0.18338608741760254 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.33230005115031613], [0.0, 0.0, 2.313316524490643]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.4\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_resu

 Completed QED-RHF in 0.0857088565826416 seconds
 Completed 1HSO Build in 2.8133392333984375e-05 seconds
 Completed ERI Build in 0.0011527538299560547 seconds 
 Completed 2D build in 7.915496826171875e-05 seconds
 Completed 1G build in 1.52587890625e-05 seconds
 Completed determinant list in 0.00043582916259765625 seconds 
 Completed constant offset matrix in 0.00014019012451171875 seconds
 Completed Hamiltonian build in 0.18364906311035156 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.35942658593809707], [0.0, 0.0, 2.5021586897551855]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.5142857142857142\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'r

 Completed QED-RHF in 0.08862709999084473 seconds
 Completed 1HSO Build in 2.8133392333984375e-05 seconds
 Completed ERI Build in 0.0011539459228515625 seconds 
 Completed 2D build in 7.486343383789062e-05 seconds
 Completed 1G build in 1.52587890625e-05 seconds
 Completed determinant list in 0.00045013427734375 seconds 
 Completed constant offset matrix in 0.0001780986785888672 seconds
 Completed Hamiltonian build in 0.184190034866333 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.386553120725878], [0.0, 0.0, 2.6910008550197277]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.6285714285714286\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_r

 Completed QED-RHF in 0.08968806266784668 seconds
 Completed 1HSO Build in 2.7894973754882812e-05 seconds
 Completed ERI Build in 0.0012700557708740234 seconds 
 Completed 2D build in 0.00011277198791503906 seconds
 Completed 1G build in 1.4066696166992188e-05 seconds
 Completed determinant list in 0.00043082237243652344 seconds 
 Completed constant offset matrix in 0.00015211105346679688 seconds
 Completed Hamiltonian build in 0.18381476402282715 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.41367965551365876], [0.0, 0.0, 2.879843020284269]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.7428571428571429\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.1208

 Completed QED-RHF in 0.0892949104309082 seconds
 Completed 1HSO Build in 2.6941299438476562e-05 seconds
 Completed ERI Build in 0.0012428760528564453 seconds 
 Completed 2D build in 8.797645568847656e-05 seconds
 Completed 1G build in 1.430511474609375e-05 seconds
 Completed determinant list in 0.0004363059997558594 seconds 
 Completed constant offset matrix in 0.00013780593872070312 seconds
 Completed Hamiltonian build in 0.18345999717712402 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4408061903014398], [0.0, 0.0, 3.0686851855488126]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.8571428571428572\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 

 Completed QED-RHF in 0.08889102935791016 seconds
 Completed 1HSO Build in 2.8133392333984375e-05 seconds
 Completed ERI Build in 0.0011639595031738281 seconds 
 Completed 2D build in 7.295608520507812e-05 seconds
 Completed 1G build in 1.4066696166992188e-05 seconds
 Completed determinant list in 0.00043010711669921875 seconds 
 Completed constant offset matrix in 0.0001652240753173828 seconds
 Completed Hamiltonian build in 0.18511676788330078 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.4679327250892207], [0.0, 0.0, 3.2575273508133544]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 1.9714285714285715\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}

 Completed QED-RHF in 0.08860206604003906 seconds
 Completed 1HSO Build in 2.8133392333984375e-05 seconds
 Completed ERI Build in 0.0012297630310058594 seconds 
 Completed 2D build in 9.012222290039062e-05 seconds
 Completed 1G build in 1.4781951904296875e-05 seconds
 Completed determinant list in 0.00043892860412597656 seconds 
 Completed constant offset matrix in 0.00015807151794433594 seconds
 Completed Hamiltonian build in 0.1883678436279297 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.49505925987700156], [0.0, 0.0, 3.446369516077897]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 2.085714285714286\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086},

 Completed QED-RHF in 0.09088325500488281 seconds
 Completed 1HSO Build in 2.6941299438476562e-05 seconds
 Completed ERI Build in 0.0011682510375976562 seconds 
 Completed 2D build in 7.295608520507812e-05 seconds
 Completed 1G build in 1.4066696166992188e-05 seconds
 Completed determinant list in 0.0004329681396484375 seconds 
 Completed constant offset matrix in 0.0001010894775390625 seconds
 Completed Hamiltonian build in 0.18785619735717773 seconds
{'repo': {'repository_url': 'https://github.com/mapol-chem/qed-ci', 'branch': 'photon_number_basis', 'commit': 'd353e5f3c747a9d747afb3934f418586cd78de17'}, 'molecule': {'molecule_name': 'LiH', 'geometry': [[0.0, 0.0, -0.5221857946647827], [0.0, 0.0, 3.6352116813424398]], 'symbols': ['Li', 'H'], 'z-matrix': '\nLi\nH 1 2.2\nsymmetry c1\n'}, 'driver': 'energy', 'model': {'method': 'qed-fci', 'orbital_basis': 'sto-3g', 'photon_basis': 'number_basis', 'number_of_photon_states': 10, 'lambda': [0.0, 0.0, 0.05], 'omega': 0.12086}, 'return_result